In [1]:
import sys
sys.path.append("D:\\WETrak")

import os
import torch
from torch.utils.data import DataLoader

import import_ipynb
import branch_adapter.regression.regression_model_utils as utils
from branch_adapter.regression.regression_network import MLP_regression
from branch_adapter.regression.regression_data import regression_data

os.environ['KMP_DUPLICATE_LIB_OK']='True'

importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_model_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_network.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_data.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_data_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_cascade_inference.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_network.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_model_utils.ipynb


In [2]:
def get_regression_inference(data_path,output_path,user_name,split,finger_type_list,classify_num,inference_mode):
    
    # -------------------------------------Save Dir initialization------------------------------------- #
    runs_dir = os.path.join(output_path, user_name,"regression",str(classify_num))
    checkpoint_dir = os.path.join(runs_dir, 'checkpoints')
    print("load regression checkpoingts from {}".format(os.path.abspath(checkpoint_dir)))

    # -------------------------------------Gpu Device--------------------------------------------------- #
    global device
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # -------------------------------------Model Initialize------------------------------------ #
    model = MLP_regression(output_size = 2*4*len(finger_type_list)).cuda()
    
    # -------------------------------------Data Initialize----------------------------------------- #
    test_set = regression_data(data_path, user_name, split, finger_type_list,classify_num,output_path,inference_mode)
    test_loader = DataLoader(test_set,batch_size=len(test_set), shuffle = False)
    
    # -------------------------------------Inference--------------------------------------------------- #
    checkpoint = utils.load_checkpoint(os.path.join(checkpoint_dir),"mean")
    model.load_state_dict(checkpoint['model_state_dict'])

    with torch.no_grad():
        model.eval()
        
        for i,(emg,_,bin_lower_limit,bin_upper_limit) in enumerate(test_loader):
            emg = utils.data_gpu(emg, device) # [B,1000,2]
            out = model(emg) # [B,8]
            
            out = out.cpu() * (bin_upper_limit-bin_lower_limit)+bin_lower_limit
            out = utils.data_gpu(out, device)

    pred_min_vals = out[:,:int(out.shape[1]/2)].cpu().numpy()
    pred_max_vals = out[:,int(out.shape[1]/2):].cpu().numpy()
    
    return pred_min_vals,pred_max_vals